# Install dependencies

In [ ]:
%pip install -q pandas pyarrow ftfy seaborn matplotlib

# 1) Paths 

In [ ]:
# Imports and paths
import os, re, json, unicodedata
from pathlib import Path
from datetime import datetime
from typing import Dict, Any

import numpy as np
import pandas as pd

SEED = 42
np.random.seed(SEED)

# Paths (works whether you run from notebooks/ or project root)
PROJ = Path.cwd().resolve().parents[0] if Path.cwd().name == 'notebooks' else Path.cwd()
DATA = PROJ / "data"
RAW = DATA / "raw"
PROC = DATA / "processed"
MODELS = PROJ / "models"
REPORTS = PROJ / "reports"
FIGS = REPORTS / "figures"
for p in [RAW, PROC, MODELS, REPORTS, FIGS]: p.mkdir(parents=True, exist_ok=True)

def timestamp():
    return datetime.utcnow().strftime("%Y%m%d_%H%M%S")

def latest_file(folder: Path, pattern="*.parquet"):
    files = sorted(folder.glob(pattern), key=lambda p: p.stat().st_mtime)
    return files[-1] if files else None

RUN_ID = timestamp()
RUN_ID

# 2) Parameters

In [ ]:
# Language filter: keep only these languages if not None
LANGS_KEEP = ["en"]  # set to None to keep all languages

# Content filters
KEEP_RETWEETS = False
KEEP_QUOTES = True
KEEP_REPLIES = True

# Minimum cleaned text length (characters)
MIN_CHAR_LEN = 3

# Heuristic to drop URL-only or almost-URL tweets
DROP_URL_HEAVY = True
URL_HEAVY_THRESHOLD = 0.7  # fraction of characters that are part of URLs

# Optional date filtering (UTC naive)
DATE_FROM = None  # e.g., "2023-01-01"
DATE_TO = None    # e.g., "2024-12-31"

# 3) Load latest raw 

In [ ]:
raw_path = latest_file(RAW, "*.parquet")
if not raw_path:
    raise FileNotFoundError("No raw parquet found in data/raw. Run 01_data_collection first.")
print(f"Loading raw snapshot: {raw_path}")

df = pd.read_parquet(raw_path)

# Basic validations
if "text" not in df.columns:
    raise ValueError("Expected 'text' column not found in raw data.")
if "created_at" in df.columns:
    df["created_at"] = pd.to_datetime(df["created_at"], errors="coerce")
rows_initial = len(df)
rows_initial

# 4) Cleaning helpers

In [ ]:
import ftfy

URL_RE = re.compile(r"(https?://\S+|www\.\S+)", flags=re.IGNORECASE)
MENTION_RE = re.compile(r"@\w+")
WHITESPACE_RE = re.compile(r"\s+")

def strip_urls(s: str) -> str:
    return URL_RE.sub("", s)

def clean_text(s: str) -> str:
    if not isinstance(s, str):
        return ""
    s = ftfy.fix_text(s)
    s = unicodedata.normalize("NFKC", s)
    # Remove URLs, keep words around them
    s = strip_urls(s)
    # Remove mentions
    s = MENTION_RE.sub("", s)
    # Remove hash sign but keep the word (e.g., #AI -> AI)
    s = s.replace("#", "")
    # Collapse whitespace
    s = WHITESPACE_RE.sub(" ", s).strip()
    return s

def url_char_fraction(original: str) -> float:
    if not isinstance(original, str) or not original:
        return 0.0
    matches = URL_RE.findall(original)
    total_url_chars = sum(len(m) for m in matches)
    return total_url_chars / max(1, len(original))

def stage_diff(before: int, after: int, name: str, tracker: Dict[str, Any]):
    tracker[name] = {"removed": before - after, "before": before, "after": after}

# 5) Apply cleaning and filters

In [ ]:
removals = {}

# Standardize columns that might be missing
for col, default in [
    ("is_retweet", False),
    ("is_quote", False),
    ("is_reply", False),
]:
    if col not in df.columns:
        df[col] = default

# Drop rows with missing/empty text
b = len(df)
df = df[df["text"].notna()]
df["text"] = df["text"].astype(str).str.strip()
df = df[df["text"].str.len() > 0]
stage_diff(b, len(df), "drop_empty_text", removals)

# Language filter (use existing 'lang' column if present)
if LANGS_KEEP is not None:
    if "lang" in df.columns:
        b = len(df)
        df = df[df["lang"].isin(LANGS_KEEP)]
        stage_diff(b, len(df), "filter_language", removals)
    else:
        print("LANGS_KEEP set but 'lang' column not found. Skipping language filter.")

# Drop retweets/quotes/replies per params
if not KEEP_RETWEETS and "is_retweet" in df.columns:
    b = len(df)
    df = df[~df["is_retweet"]]
    stage_diff(b, len(df), "filter_retweets", removals)

if not KEEP_QUOTES and "is_quote" in df.columns:
    b = len(df)
    df = df[~df["is_quote"]]
    stage_diff(b, len(df), "filter_quotes", removals)

if not KEEP_REPLIES and "is_reply" in df.columns:
    b = len(df)
    df = df[~df["is_reply"]]
    stage_diff(b, len(df), "filter_replies", removals)

# Date filters if provided
if DATE_FROM and "created_at" in df.columns:
    b = len(df)
    df = df[df["created_at"] >= pd.to_datetime(DATE_FROM)]
    stage_diff(b, len(df), "filter_date_from", removals)

if DATE_TO and "created_at" in df.columns:
    b = len(df)
    df = df[df["created_at"] <= pd.to_datetime(DATE_TO)]
    stage_diff(b, len(df), "filter_date_to", removals)

# Clean text
df["text_clean"] = df["text"].map(clean_text)

# Heuristic removal for URL-heavy posts
if DROP_URL_HEAVY:
    b = len(df)
    frac = df["text"].map(url_char_fraction)
    df = df[frac < URL_HEAVY_THRESHOLD]
    stage_diff(b, len(df), "filter_url_heavy", removals)

# Length-based filtering on cleaned text
df["char_len"] = df["text_clean"].str.len()
b = len(df)
df = df[df["char_len"] >= MIN_CHAR_LEN]
stage_diff(b, len(df), "filter_min_char_len", removals)

# Word count feature (optional, useful later)
df["word_count"] = df["text_clean"].str.split().apply(len)

# Deduplicate
b = len(df)
if "tweet_id" in df.columns:
    df = df.drop_duplicates(subset=["tweet_id"])
stage_diff(b, len(df), "dedupe_tweet_id", removals)

# Secondary dedupe on username + normalized text (case-insensitive)
df["text_norm"] = df["text_clean"].str.lower()
b = len(df)
subset_cols = ["username", "text_norm"] if "username" in df.columns else ["text_norm"]
df = df.drop_duplicates(subset=subset_cols)
stage_diff(b, len(df), "dedupe_username_text", removals)

# Sort by time if available
if "created_at" in df.columns:
    df = df.sort_values("created_at", ascending=False).reset_index(drop=True)

rows_final = len(df)
print(f"Rows: start={rows_initial:,} -> final={rows_final:,}")
df.head(3)

# 6) Quick QA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

print("Language distribution (top 10):")
if "lang" in df.columns:
    display(df["lang"].value_counts().head(10))

print("\nType distribution:")
if "is_retweet" in df.columns and "is_quote" in df.columns:
    df["type"] = np.where(df["is_retweet"], "retweet", np.where(df["is_quote"], "quote", "original"))
    display(df["type"].value_counts())

print("\nLength stats:")
display(df[["char_len", "word_count"]].describe())

# Plot length distribution
plt.figure(figsize=(6,4))
sns.histplot(df["char_len"], bins=50, color="#4C78A8")
plt.title("Cleaned text length distribution")
plt.xlabel("characters")
plt.tight_layout()
fig_path = FIGS / f"length_distribution_clean_{RUN_ID}.png"
plt.savefig(fig_path, dpi=150)
plt.show()

print(f"Saved figure: {fig_path}")

# 7) Save processed 

In [ ]:
ts = RUN_ID
proc_path = PROC / f"x_posts_clean_{ts}.parquet"
csv_sample_path = PROC / f"x_posts_clean_sample_{ts}.csv"
manifest_path = PROC / f"manifest_clean_{ts}.json"

# Save Parquet (lists like 'urls' will be preserved by pyarrow)
df.to_parquet(proc_path, index=False)

# Also save a small CSV sample for quick inspection (first 1,000 rows)
df_csv = df.copy()
if "urls" in df_csv.columns:
    df_csv["urls"] = df_csv["urls"].apply(lambda v: json.dumps(v) if isinstance(v, (list, tuple)) else v)
df_csv.head(1000).to_csv(csv_sample_path, index=False, encoding="utf-8")

manifest = {
    "run_id": ts,
    "raw_input": str(raw_path.relative_to(PROJ)),
    "processed_output": str(proc_path.relative_to(PROJ)),
    "csv_sample": str(csv_sample_path.relative_to(PROJ)),
    "rows_initial": int(rows_initial),
    "rows_final": int(rows_final),
    "columns": df.columns.tolist(),
    "params": {
        "langs_keep": LANGS_KEEP,
        "keep_retweets": KEEP_RETWEETS,
        "keep_quotes": KEEP_QUOTES,
        "keep_replies": KEEP_REPLIES,
        "min_char_len": MIN_CHAR_LEN,
        "drop_url_heavy": DROP_URL_HEAVY,
        "url_heavy_threshold": URL_HEAVY_THRESHOLD,
        "date_from": DATE_FROM,
        "date_to": DATE_TO,
    },
    "removals": removals,
    "created_at_utc": datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S"),
    "time_range": {
        "min_created_at": (df["created_at"].min().strftime("%Y-%m-%d %H:%M:%S")
                           if "created_at" in df.columns and df["created_at"].notna().any() else None),
        "max_created_at": (df["created_at"].max().strftime("%Y-%m-%d %H:%M:%S")
                           if "created_at" in df.columns and df["created_at"].notna().any() else None),
    }
}

with open(manifest_path, "w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2)

print(f"Saved processed snapshot:\n- {proc_path}\n- sample CSV: {csv_sample_path}\nManifest: {manifest_path}")